# 0-application-walk-thourgh

In this notebook, we introduce the basics of PyLipID via a walk-through application. We keep the details to minimum and invite users to refer to other specific notebooks in which you can find more details, exercise and theory. 

In this tutorial, we show the PyLipID workflow of checking the lipid interactions a Class A GPCR, Adenosine A2a receptor from coarse-grained simulations with a complex membrane bilayer that contains 7 lipid species. 

<img src="statics/A2aR_system_overview.png" />

The simulation ensemble reported in the original paper contained 10 repeats, but here we took 2 of the repeats for demonstration and saved in two directories ``run1`` and ``run2``, both of which contains a simulation trajectory ``protein_lipids.xtc`` and a corresponding topology coordinate ``protein_lipids.gro``. The two simulation trajectories have the same topology but different initial configurations. The trajectories were saved with a 20 ns timestep. 

In [1]:
%matplotlib inline
import pylipid
from pylipid.api import LipidInteraction

## Load data

We start by initializing the main class `LipidInteraction`. 

Trajectories and topology coordinates of the two repeats were saved in the directory ``../../tests/data``. Thus we first create a `trajfile_list` and `topfile_list`. Then, we load the `trajfile_list` and `topfile_list` to `LipidInteraction` together with a couple of calculation settings, including what cutoffs to use, which lipid to check, which lipid atoms are used to define the interactions and what time unit the generated data use etc. You can also specify which directory the generated data should be saved at. But here we will use the default value, i.e. the current working directory. 

In [2]:
trajfile_list = ["../../tests/data/run1/protein_lipids.xtc", "../../tests/data/run2/protein_lipids.xtc"]
topfile_list = ["../../tests/data/run1/protein_lipids.gro", "../../tests/data/run2/protein_lipids.gro"]
lipid = "CHOL"
cutoffs = [0.55, 0.8]
nprot = 1
timeunit = 'us' # micro-second

# initialize 
li = LipidInteraction(trajfile_list, topfile_list, cutoffs=cutoffs, lipid=lipid, nprot=nprot)

## Collect interactions for residues

Calculate interaction durations, occupancy and lipid count from trajectories. The correlation coefficient matrix will also be calculated from these interactions. A pandas.DataFrame object will be created to store the generated data and be stored as a class instance object `dataset`. 

In [3]:
li.collect_residue_contacts(write_log=True, print_log=False)

# print_log=True will print out the residues with the highest interaction durations, interaction 
# occupancy and lipid count from each trajectory. 

# write_log=True will write such information in a log file. 

COLLECT INTERACTIONS FROM TRAJECTORIES: 100%|██████████| 2/2 [00:49<00:00, 24.77s/it]


After calculation, a couple of class instance objects are generated. For example, the instance object `durations` stores the durations of lipid interaction with protein residues in a dictionary object. 

In [4]:
residue_id = 20
print(li.durations[residue_id])

[[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.04, 0.04, 0.04, 0.16, 0.26], [0.02, 0.02, 0.02, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.18, 0.26, 0.38]]


It shows cholesterol interaction with the 21th residue (index starting from 0) in a list of two lists, each of which contains interactions durations from the corresponding trajectory in `trajfile_list`. 

A class instance object `dataset` is also generated, which stores the averages of various interaction metrics for each residues. The information in this `dataset` will be expanded to include that of the binding site after the calculation of binding site interaction has been carried out. 

In [5]:
li.dataset.head()

,Residue,Residue ID,Occupancy,Occupancy std,Duration,Duration std,Lipid Count,Lipid Count std
0,1ILE,0,48.007968,2.191235,0.040816,0.033473,1.087129,0.000173
1,2MET,1,14.143426,0.597610,0.030566,0.021493,1.042925,0.015898
2,3GLY,2,6.374502,0.000000,0.037895,0.038331,1.000000,0.000000
3,4SER,3,1.992032,1.992032,0.048571,0.070797,0.500000,0.500000
4,5SER,4,48.207171,8.366534,0.056757,0.090585,1.077817,0.027817


The `LipidInteraction` class also has a couple of assisting functions that provide quick access to the class attributes. 

## Calculate koff/residence time for residues

We next move to calculate the interaction koff and residence time for protein residues. This calculation has to be run after ``collect_residue_contacts()`` as the koff is calculated from interaction durations. 

In [6]:
koffs, res_times = li.compute_residue_koff(print_data=False, plot_data=True)

# if you are interested in only a couple of residues or one specific residue, you can provide a list
# of residue id or one residue id as follow:
# li.compute_residue_koff(residue_id=[10,15,25,35])
# li.compute_residue_koff(residue_id=10)

`print_data=True` will print out the calculated interactions for the residues. It can be quite verbose.

`plot_data=True` will plot the koff figure for the residues. The directory to save such figures can be changed by save_dir, otherwise they will be saved in the directory of the Residue_koff_{lipid} at the current working directory. 

The koff figure looks like this:

<img src="statics/koff_figure.png" />

The left panel plots the interactions durations in the sorted order and the right panel plots the normalised survival rates of these interactions (the purple doted line) and the bi-exponential curve fitted to survival curve (red broken line). To check the sampling quality, the fitted curve of the bootstraped interaction durations are plotted in gray lines. 

## Calculate binding sites

Before calculation of interactions with binding sites, we need to calculate where the binding sites are. 

In [9]:
node_list = li.compute_binding_nodes(threshold=4, print_data=False)

# threshold=4 decides that binding sites should contain at least 4 residues. 
# print_data=True will print the residues for each binding site. It can be quite verbose. 

In [10]:
print(node_list)

[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 16, 265, 268, 272, 276, 279], [3, 7, 11, 14, 15, 18, 53, 54, 57, 58, 61, 64, 65], [19, 20, 22, 23, 26, 30, 282, 286, 293, 294, 296, 297, 298, 299, 300, 301], [25, 28, 29, 35, 40, 43, 44, 47, 48, 50, 51, 55, 80, 83, 84, 116, 119, 120, 122, 123, 126, 127, 130], [41, 90, 91, 93, 94, 96, 97, 98, 100, 101, 104, 105, 107, 108, 109, 111, 112, 113, 114, 117, 121, 124, 125, 128, 129, 132, 177, 181, 182, 185, 186, 189, 190, 193, 207], [59, 62, 67, 69, 70, 73, 74, 76, 77], [72, 131, 134, 135, 136, 137, 138, 139, 140, 173], [175, 176, 179, 180, 183, 184, 187, 188, 191, 192, 194, 195, 196, 198, 199, 201, 202, 203, 226, 233, 236, 237, 239, 240, 243, 244, 251, 254, 255], [227, 230, 231, 234, 235, 238, 242, 273, 277, 280, 281, 283, 284, 287, 288], [241, 245, 246, 248, 249, 252, 253, 256, 259, 260, 262, 263, 266, 267, 269, 270]]


This function returns a node list, which is also retored in the class instance object node_list. The node list is a list of lists, each of which contains residues for a binding site.  

## Calculate interactions with binding sites

Once binding sites are defined, we can move to the calculation of interactions with binding sites. 

In [11]:
koffs_bs, res_time_bs = li.compute_site_koff(print_data=False, plot_data=True, sort_residue="Residence Time")

CALCULATE INTERACTIONS FOR BINDING SITES: 100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


`print_data=True` will print out information for binding sites in which the comprising residues are sorted based on the value defined by `sort_residue`. Such information can be written out in a log file by `li.write_site_info()`. Similar to `compute_residue_koff()`, `compute_site_koff` also can plot the koff figure for binding sites, controled by `plot_data`. Specific binding sites can also be selected for calculated via `binding_site_id`. 

`koffs_bs` stores koffs for binding sites in a list and `res_time_bs` stores residence times for binding sites ina list. 

In [13]:
print(koffs_bs)

[3.045836950527779, 2.389003330062132, 9.442140102579902, 2.4055879456074045, 4.663923639120845, 4.2258884250499875, 6.465134977362397, 7.24927898237628, 3.7054879241121697, 1.2962624878052542]


In [14]:
li.compute_site_koff(binding_site_id=2, print_data=True, plot_data=False, sort_residue="Residence Time")

CALCULATE INTERACTIONS FOR BINDING SITES: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s]

# Binding site 2
 Binding Site Residence Time:       0.106 us   
 Binding Site Koff:                 9.442  R squared:  0.9993
 Binding Site Duration:             0.059 us   
 Binding Site Occupancy:           88.048 %
 Binding Site Lipid Count:          1.769
 Pos. Charge:        4/16      
 Neg. Charge:        0/16      
 Polar:              1/16      
 Hydrophobic:        9/16      
 Special:            2/16      
 Residue Res ID  Res. Time (us)  Duration (us)   Occupancy (%) Lipid Count  Koff  R Squared 
  21GLY    20        0.218           0.078          15.538        1.010     4.598   0.998   
  24LEU    23        0.164           0.113          50.598        1.074     6.081   1.000   
 302ARG    301       0.149           0.045          39.044        1.062     6.702   1.000   
 294ARG    293       0.145           0.044           7.570        1.000     6.882   0.999   
  27TRP    26        0.120           0.075          48.207        1.103     8.361   0.999   
 300ILE    299       

(9.442179815370299, 0.10590774795160819)

The binding site information can also be viewed by the class attribute:

In [15]:
df = li.binding_site(binding_site_id=0)

# Binding site 0
 Binding Site Residence Time:       0.328 us   
 Binding Site Koff:                 3.046  R squared:  0.9993
 Binding Site Duration:             0.087 us   
 Binding Site Occupancy:           99.602 %
 Binding Site Lipid Count:          3.098
 Pos. Charge:        0/17      
 Neg. Charge:        1/17      
 Polar:              3/17      
 Hydrophobic:        12/17     
 Special:            1/17      
 Residue Res ID  Res. Time (us)  Duration (us)   Occupancy (%) Lipid Count  Koff  R Squared 
  5SER      4        0.385           0.057          48.207        1.078     2.598   0.995   
 269TYR    268       0.370           0.230           1.992        0.500     2.701   0.973   
  14ILE    13        0.262           0.084          65.538        1.052     3.818   0.999   
  10VAL     9        0.241           0.073          83.665        1.326     4.147   0.999   
  7TYR      6        0.241           0.080          50.398        1.059     4.154   0.997   
 273VAL    272       

The returned `df` is a pandas.DataFrame object that contains the residues of the selected binding site and their interaction data. 

In [16]:
df.head()

,Residue,Residue ID,Occupancy,Occupancy std,Duration,Duration std,Lipid Count,Lipid Count std,Koff,Residence Time,R Squared,Koff Bootstrap avg,Binding Site ID,Binding Site Koff,Binding Site Residence Time,Binding Site R Squared,Binding Site Duration,Binding Site Occupancy,Binding Site Lipid Count
0,1ILE,0,48.007968,2.191235,0.040816,0.033473,1.087129,0.000173,27.501646,0.036361,0.999844,26.991207,0.0,3.045837,0.328317,0.999281,0.087488,99.601594,3.098
1,2MET,1,14.143426,0.597610,0.030566,0.021493,1.042925,0.015898,30.874937,0.032389,0.999933,30.640815,0.0,3.045837,0.328317,0.999281,0.087488,99.601594,3.098
2,3GLY,2,6.374502,0.000000,0.037895,0.038331,1.000000,0.000000,17.343492,0.057659,0.997950,25.681185,0.0,3.045837,0.328317,0.999281,0.087488,99.601594,3.098
4,5SER,4,48.207171,8.366534,0.056757,0.090585,1.077817,0.027817,2.597775,0.384945,0.995299,9.309541,0.0,3.045837,0.328317,0.999281,0.087488,99.601594,3.098
5,6VAL,5,65.737052,0.398406,0.055250,0.056369,1.124192,0.008338,14.076645,0.071040,0.999795,14.058107,0.0,3.045837,0.328317,0.999281,0.087488,99.601594,3.098


## Analyze bound poses

Once the binding sites are defined, we can also proceed to analyze bound poses, e.g. check the most representative bound poses for binding sites and check bound pose clusters for binding sites etc. 

In [17]:
pose_pool, pose_rmsd = li.analyze_bound_poses()

ANALYZE BOUND POSES: 100%|██████████| 10/10 [00:11<00:00,  1.11s/it]


`analyze_bound_poses()` will rate the bound poses for each binding site based on probability density calculated from the simulation trajectories and write out a couple of top rated poses (controled by `n_poses`). It will also cluster the bound poses for each binding site, for which the number of clusters can be determined by a density based cluster `DBSCAN` or provided via `n_clusters`. The lipid bound poses for each binding site will be returned. The RMSD of bound poses to the binding site average will be returned and plotted in a violin plot. 

<img src="statics/Pose_RMSD_violinplot.png" />

Similar to other analysis functions, `analyze_bound_poses` can also select a couple of binding sites for analysis via `binding_site_id`. 

The returned pose_pool is a `dict` object that stores bound poses for binding sites. psoe_rmsd is a pandas.DataFrame that stores RMSD for the bound poses (in unit of nm).

In [18]:
pose_rmsd.head()

,Binding Site 0,Binding Site 1,Binding Site 2,Binding Site 3,Binding Site 4,Binding Site 5,Binding Site 6,Binding Site 7,Binding Site 8,Binding Site 9
0,0.423603,0.453347,0.373516,0.626963,0.465590,0.293760,0.163058,0.478815,0.437988,0.407428
1,0.953012,0.508066,0.717344,0.492311,0.532772,0.542467,0.177871,0.523724,0.501309,0.522759
2,0.372235,0.559859,0.344610,0.357748,0.797744,0.393510,0.202087,0.701567,0.760275,0.712557
3,0.399955,0.505883,0.894511,0.648025,0.587807,0.231317,0.132604,0.375940,0.440442,0.728238
4,0.410028,0.421467,0.260160,0.503571,0.542116,0.509176,0.249426,0.463891,0.437455,0.362353


## Calculate surface area

We can also calculate the surface area for binding site, obtaining the values as a function of time. 

In [19]:
surface_area = li.compute_surface_area(plot_data=True)

CALCULATE SURFACE AREA PER TRAJ: 100%|██████████| 2/2 [00:19<00:00,  9.77s/it]


The returned `surface_area` is pandas.DataFrame object that stores the time evolution of the surface area for each binding site. The reported values are in the unit of nm^2.

In [20]:
surface_area.head()

Binding Site 0  Binding Site 1  Binding Site 2  Binding Site 3  \
0 0 0       11.121536        6.162645       13.373794       11.226026   
    1       10.852450        6.518453       13.187954       11.928950   
    2       11.168357        5.824062       13.586670       11.146304   
    3       11.250794        6.969460       12.726676       12.118644   
    4       10.486482        6.094445       13.585187       13.025974   

       Binding Site 4  Binding Site 5  Binding Site 6  Binding Site 7  \
0 0 0       20.109919        5.424675        5.639021       18.713718   
    1       20.201431        5.231493        5.839314       18.968510   
    2       19.739061        5.281025        6.109618       18.965055   
    3       20.140154        5.381027        6.064853       18.926474   
    4       20.513193        5.486375        5.845545       18.649933   

       Binding Site 8  Binding Site 9  Time  
0 0 0        9.377054       10.777733  3.00  
    1       10.264684       10.718760  3.02  
    2       10.388362       10.001993  3.04  
    3        9.435140       10.028803  3.06  
    4        9.907309        9.405209  3.08

`print_data=True` generates a timeseries plot and a violinplot. 

<img src="statics/Surface_Area_timeseries.png" />

<img src="statics/Surface_Area_violinplot.png" />

Similar to other binding site analysis functions, `compute_surface_area` allows for selection of binding site for calculation via `binding_site_id`.


The calculation of surface area needs definition of the radii of atoms in the system. The function uses `mdtraj`'s atom radii and defines the radii of MARTINI 2 beads. New radii definition can be provided via `radii`. 

## Plots and save data

The `LipidInteraction` provides a couple of assisting functions to plot and save data. 

In [21]:
for item in ["Residence Time", "Duration", "Occupancy", "Lipid Count"]:
    li.plot(item=item) # plot values as a function of time

<img src="statics/Residence_Time.png" />

<img src="statics/Duration.png" />

<img src="statics/Occupancy.png" />

<img src="statics/Lipid_Count.png" />

In [23]:
li.plot_logo(item="Residence Time")

<img src="statics/Residence_Time_logo.png" />

In [22]:
li.write_site_info(sort_residue="Duration") # write binding site information in a txt file

li.save_data(item="Dataset") # save the pandas.DataFrame object dataset in a csv file

li.save_data(item="Duration") # save all the interaction durations in pickle

li.save_coordinate(item="Residence Time") # write protein coordinate in pdb format with the b factor column
                                          # showing 'Residence Time' values

li.save_pymol_script(pdb_file="../../tests/data/receptor.pdb") # write a pymol script that maps binding 
                                                               # site information to protein structure. 